In [4]:
import numpy as np
import time

# Função _b que simula a medição do tempo de execução
def _b(data, kernel, repeat, timer, is_ns_timer, resolution):
    start = timer()  # Inicia o temporizador
    for _ in range(repeat):
        kernel(data)  # Executa a função que você está medindo
    end = timer()  # Finaliza o temporizador
    total_time = end - start
    return total_time / repeat, total_time  # Retorna o tempo por repetição e o tempo total

# Exemplo de kernel para ser testado
def kernel_example(data):
    return sum(data)  # Simplesmente soma os elementos do dado

# Parâmetros de teste
data = np.random.rand(100)  # Exemplo de dados aleatórios
repeat = 5  # Número inicial de repetições
timer = time.perf_counter  # Usa perf_counter para medir o tempo
is_ns_timer = False  # Se você quiser usar nanosegundos, depende do seu timer
resolution = 1e-7  # Resolução do temporizador
n_range = [2**k for k in range(2)]  # Faixa de tamanhos de entrada
timings = np.zeros((1, len(n_range)))  # Matriz para armazenar os tempos
remaining_time = 10  # Tempo máximo para rodar o experimento (10 segundos)

# Implementação da lógica principal
try:
    for i, n in enumerate(n_range):
        t, total_time = _b(data[:n], kernel_example, repeat, timer, is_ns_timer, resolution)
        time_per_repetition = total_time / repeat

        remaining_time -= total_time
        repeat = int(remaining_time // time_per_repetition)  # Ajusta o número de repetições
        if repeat > 0:
            t2, _ = _b(data[:n], kernel_example, repeat, timer, is_ns_timer, resolution)
            t = min(t, t2)

        timings[0, i] = t  # Armazena o tempo

except KeyboardInterrupt:
    timings = timings[:, :i]
    n_range = n_range[:i]

# Exibe o resultado
print("Tamanhos de entrada (n_range):", n_range)
print("Tempos medidos (timings):", timings)


Tamanhos de entrada (n_range): [1, 2, 4]
Tempos medidos (timings): [[4.71250034e-07 5.12098038e-07 6.28922679e-07]]


In [11]:
import optuna
import numpy as np
import pandas as pd

#criando estudo pra comparar desempenho entre np e pandas
def objective_np(trial,input_data):
    n = trial.suggest_int(1e3, 1e6, 1e9)
    data = np.random.rand(n)
    repeat = 5
    timer = time.perf_counter
    is_ns_timer = False
    resolution = 1e-7
    t, _ = _b(data, kernel_example, repeat, timer, is_ns_timer, resolution)
    return t

def objective_pd(trial):
    n = trial.suggest_int(1e3, 1e6, 1e9)
    data = pd.Series(n)
    repeat = 5
    timer = time.perf_counter
    is_ns_timer = False
    resolution = 1e-7
    t, _ = _b(data, kernel_example, repeat, timer, is_ns_timer, resolution)
    return t

study = optuna.create_study()
input_data = np.random.rand(1e12)
study.optimize(objective, n_trials=10)
print(study.best_params)

[I 2024-09-24 09:04:58,284] A new study created in memory with name: no-name-aa3306fe-7b10-4cfc-ac52-9adf37f1f8cb
[I 2024-09-24 09:04:58,375] Trial 0 finished with value: 0.01772814000141807 and parameters: {'n': 329775}. Best is trial 0 with value: 0.01772814000141807.
[I 2024-09-24 09:04:58,394] Trial 1 finished with value: 0.0033728200010955333 and parameters: {'n': 71103}. Best is trial 1 with value: 0.0033728200010955333.
[I 2024-09-24 09:04:58,499] Trial 2 finished with value: 0.020314840000355616 and parameters: {'n': 401522}. Best is trial 1 with value: 0.0033728200010955333.
[I 2024-09-24 09:04:58,619] Trial 3 finished with value: 0.023130300000775605 and parameters: {'n': 477035}. Best is trial 1 with value: 0.0033728200010955333.
[I 2024-09-24 09:04:58,680] Trial 4 finished with value: 0.011874899998656475 and parameters: {'n': 252065}. Best is trial 1 with value: 0.0033728200010955333.
[I 2024-09-24 09:04:58,726] Trial 5 finished with value: 0.008724839999922551 and paramet

{'n': 71103}


In [ ]:
import pandas as pd
import numpy as np
import time
import psutil
from itertools import product

# Função para criar um DataFrame de um determinado tamanho
def create_dataframe(size):
    return pd.DataFrame(np.random.random((size, 3)), columns=['A', 'B', 'C'])

# Função para medir o tempo de leitura
def measure_read_time(size, max_ram_mb, max_cpu_percent):
    # Garante que a RAM e a CPU não ultrapassem os limites especificados
    while True:
        ram_usage = psutil.virtual_memory().used / (1024 ** 2)  # RAM em MB
        cpu_usage = psutil.cpu_percent(interval=1)  # Porcentagem de uso da CPU

        if ram_usage < max_ram_mb and cpu_usage < max_cpu_percent:
            break

    # Criação do DataFrame
    df = create_dataframe(size)

    # Medir o tempo de leitura
    start_time = time.time()
    # Simulação da leitura (pode ser alterada conforme necessário)
    df_read = df.copy()  # A cópia simula a leitura
    end_time = time.time()

    return end_time - start_time  # Tempo de leitura

# Parâmetros de teste
dataset_sizes = [10**3, 10**4, 10**5]  # Tamanhos do dataset
max_ram_values = [256, 512]  # Limite máximo de RAM em MB
max_cpu_percent_values = [60, 80]  # Porcentagem máxima de uso da CPU

# Gerar todas as combinações possíveis de fatores
combinations = list(product(dataset_sizes, max_ram_values, max_cpu_percent_values))

# Coletar resultados
results = []

# Realizar o experimento para cada combinação
for size, ram_limit, cpu_limit in combinations:
    read_time = measure_read_time(size, ram_limit, cpu_limit)
    results.append((size, ram_limit, cpu_limit, read_time))
    print(f"Tamanho do Dataset: {size}, Max RAM: {ram_limit}MB, Max CPU: {cpu_limit}%, Tempo de Leitura: {read_time:.4f} segundos")

# Exibir os resultados
print("\nResultados:")
for size, ram_limit, cpu_limit, time_taken in results:
    print(f"Tamanho: {size}, Max RAM: {ram_limit}MB, Max CPU: {cpu_limit}%, Tempo de Leitura: {time_taken:.4f} segundos")


In [1]:
import pandas as pd
import numpy as np
import time
import psutil
import optuna

# Função para criar um DataFrame de um determinado tamanho
def create_dataframe(size):
    return pd.DataFrame(np.random.random((size, 3)), columns=['A', 'B', 'C'])

# Função para medir o tempo de leitura
def measure_read_time(size, max_ram_mb, max_cpu_percent):
    # Garante que a RAM e a CPU não ultrapassem os limites especificados
    while True:
        ram_usage = psutil.virtual_memory().used / (1024 ** 2)  # RAM em MB
        cpu_usage = psutil.cpu_percent(interval=40)  # Porcentagem de uso da CPU
        if ram_usage < max_ram_mb and cpu_usage < max_cpu_percent:
            break

    # Criação do DataFrame
    df = create_dataframe(size)

    # Medir o tempo de leitura
    start_time = time.time()
    # Simulação da leitura
    df_read = df.copy()  # A cópia simula a leitura
    end_time = time.time()

    return end_time - start_time  # Tempo de leitura

# Função de otimização do Optuna
def objective(trial):
    # Definindo o espaço de busca
    dataset_size = trial.suggest_categorical("dataset_size", [10**1, 10**2, 10**3])
    max_ram = trial.suggest_categorical("max_ram_mb", [1024, 2048])
    max_cpu = trial.suggest_categorical("max_cpu_percent", [60, 80])

    # Medir o tempo de leitura
    read_time = measure_read_time(dataset_size, max_ram, max_cpu)

    print(f"Tamanho do Dataset: {dataset_size}, Max RAM: {max_ram}MB, Max CPU: {max_cpu}%, Tempo de Leitura: {read_time:.4f} segundos")

    return read_time  # A variável de resposta a ser minimizada

# Executar o estudo
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # Número de tentativas

# Resultados
best_trial = study.best_trial

print("Melhor Trial:")
print(f"Tamanho do Dataset: {best_trial.params['dataset_size']}")
print(f"Max RAM: {best_trial.params['max_ram_mb']}MB")
print(f"Max CPU: {best_trial.params['max_cpu_percent']}%")
print(f"Tempo de Leitura: {best_trial.value:.4f} segundos")


[I 2024-09-24 09:51:28,725] A new study created in memory with name: no-name-33eebb16-8f80-4dba-9026-964be5f33135
[W 2024-09-24 09:54:08,750] Trial 0 failed with parameters: {'dataset_size': 10, 'max_ram_mb': 1024, 'max_cpu_percent': 60} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Python312\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ualac\AppData\Local\Temp\ipykernel_32668\756024554.py", line 39, in objective
    read_time = measure_read_time(dataset_size, max_ram, max_cpu)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ualac\AppData\Local\Temp\ipykernel_32668\756024554.py", line 16, in measure_read_time
    cpu_usage = psutil.cpu_percent(interval=40)  # Porcentagem de uso da CPU
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ualac\AppData\Roaming\Python\Python312\site

KeyboardInterrupt: 

In [3]:
import pandas as pd
import numpy as np
import time
import psutil
from itertools import product

# Função para criar um DataFrame de um determinado tamanho
def create_dataframe(size):
    return pd.DataFrame(np.random.random((size, 3)), columns=['A', 'B', 'C'])

# Função para medir o tempo de leitura
def measure_read_time(size, max_ram_mb, max_cpu_percent):
    # Garante que a RAM e a CPU não ultrapassem os limites especificados
    while True:
        ram_usage = psutil.virtual_memory().used / (1024 ** 2)  # RAM em MB
        cpu_usage = psutil.cpu_percent(interval=1)  # Porcentagem de uso da CPU

        # Muda a condição para permitir uso da CPU a partir de 20%
        if ram_usage < max_ram_mb and cpu_usage < max_cpu_percent and cpu_usage >= 20:
            break

    # Criação do DataFrame
    df = create_dataframe(size)

    # Medir o tempo de leitura
    start_time = time.time()
    # Simulação da leitura (pode ser alterada conforme necessário)
    df_read = df.copy()  # A cópia simula a leitura
    end_time = time.time()

    return end_time - start_time  # Tempo de leitura

# Parâmetros de teste
dataset_sizes = [10*1, 10**2, 10**3]  # Tamanhos do dataset
max_ram_values = [1024, 2048]  # Limite máximo de RAM em MB
max_cpu_percent_values = [60, 80]  # Porcentagem máxima de uso da CPU

# Gerar todas as combinações possíveis de fatores
combinations = list(product(dataset_sizes, max_ram_values, max_cpu_percent_values))

# Coletar resultados
results = []

# Realizar o experimento para cada combinação
for size, ram_limit, cpu_limit in combinations:
    read_time = measure_read_time(size, ram_limit, cpu_limit)
    results.append((size, ram_limit, cpu_limit, read_time))
    print(f"Tamanho do Dataset: {size}, Max RAM: {ram_limit}MB, Max CPU: {cpu_limit}%, Tempo de Leitura: {read_time:.4f} segundos")

# Exibir os resultados
print("\nResultados:")
for size, ram_limit, cpu_limit, time_taken in results:
    print(f"Tamanho: {size}, Max RAM: {ram_limit}MB, Max CPU: {cpu_limit}%, Tempo de Leitura: {time_taken:.4f} segundos")


KeyboardInterrupt: 

In [3]:
import pandas as pd
import numpy as np
import time
import psutil
from itertools import product

# Função para criar um DataFrame de um determinado tamanho
def create_dataframe(size):
    #return pd.DataFrame(np.random.random((size, 3)), columns=['A', 'B', 'C'])
    return pd.DataFrame(np.random.uniform(0, 3, (size, 3)), columns=['A', 'B', 'C'])

# Função para medir o tempo de leitura
def measure_read_time(size):
    df = create_dataframe(size)
    start_time = time.time()
    df_read = df.copy()  # A cópia simula a leitura
    df_read.groupby('A').agg({'B': 'mean', 'C': 'sum'})
    end_time = time.time()
    return end_time - start_time  # Tempo de leitura

# Parâmetros de teste
dataset_sizes = [1e6,1e7,5e7]  # Tamanhos do dataset
max_ram_values = [256,512,1024]  # Limite máximo de RAM em MB
max_cpu_percent_values = [2, 10, 50]  # Porcentagem máxima de uso da CPU

# Gerar todas as combinações possíveis de fatores
combinations = list(product(dataset_sizes, max_ram_values, max_cpu_percent_values))

# Coletar resultados
results = []

# Realizar o experimento para cada combinação
for size, ram_limit, cpu_limit in combinations:
    size = int(size)
    read_time = measure_read_time(size)  # Remover verificação de RAM e CPU
    results.append((size, ram_limit, cpu_limit, read_time))

# Criar um DataFrame com os resultados
results_df = pd.DataFrame(results, columns=['Tamanho', 'Max_RAM_MB', 'Max_CPU_Percent', 'Tempo_Leitura'])

# Exibir os resultados
print(results_df.sort_values(['Tamanho','Max_CPU_Percent']).sort_values('Tempo_Leitura', ascending=True))

# Opcional: Salvar os resultados em um arquivo CSV
#results_df.to_csv('resultados_leitura.csv', index=False)


     Tamanho  Max_RAM_MB  Max_CPU_Percent  Tempo_Leitura
1    1000000         256               10       0.347864
2    1000000         256               50       0.355353
0    1000000         256                2       0.356523
5    1000000         512               50       0.366745
7    1000000        1024               10       0.370284
6    1000000        1024                2       0.375225
8    1000000        1024               50       0.381962
4    1000000         512               10       0.387728
3    1000000         512                2       0.394789
11  10000000         256               50       5.842160
9   10000000         256                2       5.869317
13  10000000         512               10       5.870237
14  10000000         512               50       5.893767
12  10000000         512                2       5.906771
15  10000000        1024                2       5.908139
10  10000000         256               10       5.931088
16  10000000        1024       